# 1. 터미널을 통해 스파크 서버의 접속 정보를 확인합니다
```bash
bash> cd ~/workspace/data-engineer-basic-training/day5
bash> docker-compose up -d
bash> docker-compose ps

# 아래와 같이 http://127.0.0.1 로 시작하는 URL 에서 127.0.0.1 부분을 현재 aws-instance-host 이름으로 변경합니다
bash> docker-compose logs notebook | grep http
...
or http://127.0.0.1:8888/?token=ad4f43203ac46f7f7f58807ab6781b1fd18b9ca5066664df
...
```

### 1-1. 크롬을 통해서 [http://aws-hostname:8888/?token=...](http://<aws-hostname>:8888/?token=ad4f43203ac46f7f7f58807ab6781b1fd18b9ca5066664df) 에 접속합니다

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .appName("Data Engineer Basic Day5") \
    .config("spark.dataengineer.basic.day5", "step3") \
    .config("spark.sql.session.timeZone", "Asia/Seoul") \
    .getOrCreate()

# 2. 필요한 모든 데이터를 읽어서 데이터프레임으로 생성합니다

### 2-1. 각 경로에서 10/25 기준 데이터를 읽고 스키마와 데이터를 확인합니다
> 경로에서 데이터를 읽어서 TABLENAME_25 와 같이 날짜가 붙은 테이블을 생성합니다

In [ ]:
# 1. 오늘은 2020/10/26 이고, 어제가 서비스 첫 날인 2020/10/25 날짜의 기준 지표를 생성합니다

user25 = spark.read.parquet("user/20201025")
user25.createOrReplaceTempView("user25") # 테이블생성
user25.printSchema() # 스키마 출력
user25.show(truncate=False) # 데이터 확인

purchase25 = spark.read.parquet("purchase/20201025")
# 실습1) purchase25 테이블생성
# 실습) 스키마 출력
# 실습) 데이터 확인

accesslog = spark.read.option("inferSchema", "true").json("access/20201025")
accesslog.createOrReplaceTempView("accesslog")
access25 = spark.sql("select a_id, a_tag, a_time, a_timestamp, a_uid from accesslog")
# 실습) access25 테이블생성
# 실습) 스키마 출력
# 실습) 데이터 확인

### 2-2. 사용에 용이하게 하기 위해 2020/10/25 일자의 데이터만 필터하여 데이터프레임을 생성합니다
> unixtime 을 문자열 시간으로 변경하는 함수는 from_unixtime 입니다

In [ ]:
# 실습2) p_time_condition 값은 p_time 값이 '2020-10-25 00:00:00' 보다 크거나 같고, '2020-10-26 00:00:00' 보다 작은 조건입니다
p_time_condition = "p_time <> ''"

purchase = spark.sql("select from_unixtime(p_time) as p_time, p_uid, p_id, p_name, p_amount from purchase25").where(p_time_condition)
purchase.createOrReplaceTempView("purchase")
purchase.printSchema()
purchase.show()

# 실습3) u_signup_condition 값은 u_signup 값이 '20201025' 보다 크거나 같고, '20201026' 보다 작은 조건입니다
u_signup_condition = "u_signup <> ''"

user = spark.sql("select u_id, u_name, u_gender from user25").where(u_signup_condition)
user.createOrReplaceTempView("user")
user.printSchema()
user.show()

access = spark.sql("select a_id, a_tag, a_timestamp, a_uid from access25")
access.createOrReplaceTempView("access")
access.printSchema()
access.show()

# 3. Daily Activer User 지표를 생성하세요

### 3-1. access 테이블을 이용하여 DAU 지표를 생성하세요
```text
지표정의 : 지정한 일자의 접속한 유저 수
지표산식 : 지정한 일자의 접속 테이블에 로그(로그인 혹은 로그아웃)가 한 번 이상 발생한 이용자의 빈도수
입력형태 : access 테이블
출력형태 : number
```

In [ ]:
spark.sql("select a_timestamp as a_time, a_uid from access").show(truncate=False)

# 실습4) distinctAccessUser 는 access 테이블에 대해 'count(distinct column) as DAU' 명령을 이용하여 distinct user 지표를 생성합니다
distinctAccessUser = ""

dau = spark.sql(distinctAccessUser)
dau.show()

# 4. Daily Paying User 지표를 생성하세요

### 4-1. purchase 테이블을 이용하여 PU 지표를 생성하세요
```text
지표정의 : 지정한 일자의 구매 유저 수
지표산식 : 지정한 일자의 구매 테이블에 한 번이라도 구매가 발생한 이용자의 빈도수
입력형태 : purchase 테이블
출력형태 : number
```

In [ ]:
spark.sql("select p_time, p_uid from purchase").show()

# 실습5) distinctPayingUser 는 purchase 테이블에 대해 'count(distinct column) as PU' 함수를 통해 구매유저 수를 구합니다
distinctPayingUser = "" 

pu = spark.sql(distinctPayingUser)
pu.show()

# 5. Daily Revenue 지표를 생성하세요

### 5-1. purchase 테이블을 이용하여 PU 지표를 생성하세요
```text
지표정의 : 지정한 일자에 발생한 총 매출 금액
지표산식 : 지정한 일자의 구매 테이블에 저장된 전체 매출 금액의 합
입력형태 : access 테이블
출력형태 : number
```

In [ ]:
spark.sql("select p_amount as daily_revenue from purchase").show()

# 실습6) sumOfDailyRevenue 는 sum(column) as DR 함수를 통해 p_amount 의 합을 구하시오
sumOfDailyRevenue = "" 

dr = spark.sql(sumOfDailyRevenue)
dr.show()

# 6. ARPU - Average Revenue Per User 지표를 생성하세요
> 이미 생성된 데이터프레임으로부터 이름으로 하나의 값을 가져오는 방법은 아래와 같습니다
```python
변수명 = 데이터프레임.collect()[0]["이름"]
v_dau = dau.collect()[0]["DAU"]
```


### 5-1. daily revenue 와 daily user 정보를 이용하여 ARPU 를 구하세요
```text
지표정의 : 유저 당 평균 발생 매출 금액
지표산식 : 총 매출 / 전체 유저 수 = DR / DAU
입력형태 : Daily Revenue, Daily Active User
출력형태 : number
```

In [ ]:
v_dau = dau.collect()[0]["DAU"]
v_pu = pu.collect()[0]["PU"]
v_dr = dr.collect()[0]["DR"]
v_unknown = 1

# 실습7) ARPU 계산을 위한 v_unknown 를 각 각 다른 변수로 치환하세요
print("+------------------+")
print("|             ARPU |")
print("+------------------+")
print("|        {} |".format(v_unknown / v_unknown))
print("+------------------+")

# 6. ARPPU - Average Revenue Per Paying User 지표를 생성하세요
> 이미 생성된 데이터프레임으로부터 이름으로 하나의 값을 가져오는 방법은 아래와 같습니다
```python
변수명 = 데이터프레임.collect()[0]["이름"]
v_dau = dau.collect()[0]["DAU"]
```

### 6-1. daily revenue 와 daily paying user 정보를 이용하여 ARPU 를 구하세요
```text
지표정의 : 유저 당 평균 발생 매출 금액
지표산식 : 총 매출 / 전체 유저 수 = DR / DPU
입력형태 : Daily Revenue, Daily Paying User
출력형태 : number
```

In [ ]:
# 7. Average Revenue Per Paying User

# 실습8) ARPPU 계산을 위한 v_unknown 를 각 각 다른 변수로 치환하세요
print("+------------------+")
print("|            ARPPU |")
print("+------------------+")
print("|        {} |".format(v_unknown / v_unknown))
print("+------------------+")

# 7. 디멘젼 테이블을 생성하세요
> 디멘젼 테이블을 스키마는 아래와 같습니다.

| 컬럼명 | 컬럼타입 | 설명 |
| - | - | - |
| d_uid | integer | 아이디 |
| d_name | string | 이름 |
| d_pamount | integer | 누적 구매 금액 |
| d_pcount | integer | 누적 구매 횟수 |
| d_acount | integer | 누적 접속 횟수 |
| d_first_purchase | string | 최초 구매 일시 |
| dt | string | 유저아이디 |

### 7-1. 서비스 오픈 첫 날이므로 오늘 로그만 이용하여 전체 유저아이디와 접속 횟수를 구합니다
```text
지표정의 : 이용자 누적 상태 정보
지표산식 : 오늘까지 접속한 모든 유저의 정보를 저장하는 테이블
입력형태 : user, purchase, access
출력형태 : 아래와 같이 설계합니다
```

In [ ]:
access.printSchema()

# 실습9) countOfAccess 는 access 테이블로부터 a_id 가 'login' 인 a_uid 값의 빈도수를 group by a_uid 집계를 통해 구하시오
countOfAccess = "" 

uids = spark.sql(countOfAccess)
uids.show()

### 7-2. 일 별 이용자 별 총 매출 금액과, 구매 횟수를 추출합니다

In [ ]:
purchase.printSchema()

# 실습10) sumOfCountAndAmount 는 purchase 테이블로 부터 p_uid 별 매출 횟수(count)와, 매출 금액의 합(sum)을 구하는 집계 쿼리를 생성 하시오
sumOfCountAndAmount = "" 

amts = spark.sql(sumOfCountAndAmount)
amts.show()

### 7-3. 이용자 정보와 구매 정보와 조인합니다

In [ ]:
# 실습11) joinCondition 는 uids 와 amts 데이터프레임의 uid 컬럼이 일치하는 조건을 작성하시오
joinCondition = ""

# 실습12) joinHow 는 어떤 조인을 할 지 결정합니다
joinHow = ""

dim1 = uids.join(amts, joinCondition, joinHow).sort(uids["a_uid"].asc())  # 이렇게 Left Outer Join 연산으로 계속 붙여 나갑니다
dim1.printSchema()
dim1.show()

### 7-4. 고객 정보를 추가합니다

In [ ]:
# 실습13) joinCondition 는 dim1 과 user 의 uid 값이 일치하는 조건을 작성하시오
joinCondition = ""

# 실습14) joinHow 는 어떤 조인을 할 지 결정합니다
joinHow = ""

dim2 = dim1.join(user, joinCondition, joinHow)
dim2.printSchema()
dim2.show()

### 7-5. 중복되는 ID 컬럼은 제거하고, 숫자 필드에 널값은 0으로 기본값을 넣어줍니다

In [ ]:
dim3 = dim2.drop("p_uid", "u_id")

# 실습15) p_amount, p_count 필드의 기본값을 0으로 넣도록 작성합니다
fillDefaultValue = {}

dim4 = dim3.na.fill(fillDefaultValue)
dim4.printSchema()
dim4.show()

### 7-6. 생성된 유저 테이블을 재사용 가능하도록 컬럼 명을 변경합니다

In [ ]:
dim4.printSchema()

# 실습16) d_ 로 시작하는 컬럼으로 모두 rename 합니다.
# withColumnRenamed("a_uid", "d_acount") 라인을 반복해서 a_count ~ u_gender 컬럼을 rename 합니다
# a_uid -> d_uid
# a_count -> d_acount
# p_amount -> d_pamount
# p_count -> d_pcount
# u_name -> d_name
# u_gender -> d_gender

# 실습17) d_ 로 시작하지 않는 컬럼은 모두 drop 합니다
# drop   : 사용하지 않는 컬럼을 데이터프레임에서 제거합니다 - "a_uid", "a_count", "p_amount", "p_count", "u_name", "u_gender"

# 실습18) 혹은 d_ 로 시작하는 컬럼만 select 합니다
# select : 컬럼명을 변경한 이후에 d_ 로 시작하는 컬럼만 select 할 수 있도록 컬럼을 명시합니다 - "d_uid", "d_name", "d_gender", "d_acount", "d_pamount", "d_pcount"

# drop 혹은 select 어느 하나의 방식을 사용해도 됩니다
dim5 = dim4 \
.withColumnRenamed("a_uid", "d_uid") \
.drop("사용하지 않는 컬럼 목록") \
.select("사용하는 컬럼 목록")

dim5.show()

### 7-7. 최초 구매 유저 정보를 추가합니다
```bash
# 아래의 구문을 통해 특정 컬럼이 널인 경우의 값을 선택할 수 있습니다 
expr("case when column1 is null then column2 else column1 end")
```

In [ ]:
# 실습19) selectFirstPurchaseTime 는 하루에 여러번 구매가 있을 수 있으므로 가장 먼저 구매한 일시를 min 함수를 써서 선택합니다
selectFirstPurchaseTime = ""

first_purchase = spark.sql(selectFirstPurchaseTime)
dim6 = dim5.withColumn("d_first_purchase", lit(None))

# 실습20) exprFirstPurchase 는 d_first_purchase 컬럼이 널이면 p_time 을 사용하고 그렇지 않으면 d_first_purchase 컬럼을 사용하도록 expr 의 null 값을 치환하시오
exprFirstPurchase = expr("d_first_purchase")

dimension = dim6.join(first_purchase, dim5.d_uid == first_purchase.p_uid, "left_outer") \
.withColumn("first_purchase", exprFirstPurchase) \
.drop("d_first_purchase", "p_uid", "p_time") \
.withColumnRenamed("first_purchase", "d_first_purchase")

dimension.printSchema()
dimension.show()

### 7-8. 오늘 디멘젼 테이블을 dim_users/dt=20201025 경로에 저장합니다.

In [ ]:
# 실습21) dim_users/dt=20201025 경로에 저장합니다
today_dimension = ""

dimension.write.mode("overwrite").parquet(today_dimension)

### 7-9. 오늘의 주요 지표를 출력합니다

In [ ]:
print("+------------------+")
print("|              DAU |")
print("+------------------+")
print("|                {} |".format(v_dau))
print("+------------------+")
print("+------------------+")
print("|               PU |")
print("+------------------+")
print("|                {} |".format(v_pu))
print("+------------------+")
print("+------------------+")
print("|                DR |")
print("+------------------+")
print("|        {} |".format(v_dr))
print("+------------------+")
print("+------------------+")
print("|             ARPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_dau))
print("+------------------+")
print("+------------------+")
print("|            ARPPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_pu))
print("+------------------+")